In [ ]:
import pandas as pd
import numpy as np
import random
import os
from IPython.display import clear_output
from google.cloud import bigquery # SQL table interface on Arcus
import json
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Initialize the client service
client = bigquery.Client()

grader_table_name = "lab.grader_table_with_metadata_project_independent"
project_table_name = "lab.proc_ord_projects"

In [ ]:
def print_report_experimental(
    proc_id, client, to_highlight={}, source_table="arcus.procedure_order_narrative"
):
    # Get the grading info for the proc_ord_id
    q_get_grade_row = ( 
        'select * from '
        + 'lab.grader_table_with_metadata_project_independent '
        + 'where proc_ord_id = "'
        + proc_id
        + '" '
    )
    df_grade = client.query(q_get_grade_row).to_dataframe()

    for idx, row in df_grade.iterrows():
        grader = row['grader_name']
        grade = row['grade']
        grade_date = row['grade_date']
        print("Grader", grader, "gave grade", grade, "on", grade_date)

    # Get the specifics of the grade of 0
    zero_row = df_grade[df_grade['grade'] == 0]
    s_zero_row = zero_row['grader_name'].values+" gave grade 0 on " + zero_row['grade_date'].values
        
    print("Age at scan:", (df_grade['age_in_days'].values[0]/365.25))
    print()
    
    
    # Get the report for that proc_ord_id from the primary report table
    try:
        q_get_report_row = (
            "SELECT * FROM "
            + source_table
            + ' where proc_ord_id like "'
            + str(proc_id)
            + '"'
        )
        df_report = client.query(q_get_report_row).to_dataframe()
    except:
        print(
            "AN ERROR HAS OCCURRED: REPORT", proc_id, "CANNOT BE FOUND IN", source_table
        )

    # If the id was in the new table:
    if len(df_report) == 1:
        origin_table = source_table
        domain = source_table.split(".")[0]

        q_get_report_row = (
            "SELECT * FROM "
            + domain
            + '.procedure_order_narrative where proc_ord_id = "'
            + str(proc_id)
            + '"'
        )
        report_text = (
            client.query(q_get_report_row).to_dataframe()["narrative_text"].values[0]
        )

        q_get_report_row = (
            "SELECT * FROM "
            + domain
            + '.procedure_order_impression where proc_ord_id = "'
            + str(proc_id)
            + '"'
        )
        df_report = client.query(q_get_report_row).to_dataframe()

        if len(df_report) == 1:
            report_text += "\n\nIMPRESSION: " + df_report["impression_text"].values[0]

    elif len(df_report) == 0:
        print("proc_ord_id not in", source_table, ":", proc_id)

    report_text = " ".join(report_text.split())
    report_text = report_text.replace("CLINICAL INDICATION", "\n\nCLINICAL INDICATION")
    report_text = report_text.replace("TECHNIQUE", "\n\nTECHNIQUE")
    report_text = report_text.replace("HISTORY", "\n\nHISTORY")
    report_text = report_text.replace("IMPRESSION", "\n\nIMPRESSION")
    report_text = report_text.replace("FINDINGS", "\n\nFINDINGS")
    report_text = report_text.replace("COMPARISON", "\n\nCOMPARISON")

    # If the user passed a dictionary of lists to highlight
    if len(to_highlight.keys()) > 0:
        for key in to_highlight.keys():
            report_text = mark_text_color(report_text, to_highlight[key], key)

    # Print the report and ask for a grade
    print(report_text)
    print()
    # Print the proc_ord_id
    print("Report id:", str(proc_id))
    print()
    
    return s_zero_row

In [ ]:
def mark_text_color(line, to_mark, color):
    # Sort the list of strings to highlight by length (longest to shortest)
    to_mark = sorted(to_mark, key=len)[::-1]

    if color == "green":
        start = "\x1b[5;30;42m"  # green background, bold black text
    elif color == "yellow":
        start = "\x1b[5;30;43m"  # yellow background, bold black text
    elif color == "red":
        start = "\x1b[5;30;41m"  # red background, bold black text
    elif color == "gray" or color == "grey":
        start = "\x1b[5;30;47m"  # gray background, bold black text

    end = "\x1b[0m"

    if line is np.nan:
        return "<No report available.>"

    if type(to_mark) == str:
        line = line.replace(to_mark, start + to_mark + end)

    elif type(to_mark) == list:
        for phrase in to_mark:
            line = line.replace(str(phrase), start + str(phrase).upper() + end)

    else:
        print("Error: the second argument must be either a string or a list of strings")

    return line

In [ ]:
q_conflicting_reports = '''
select
  grades.proc_ord_id,
  grades.grade,
  nlp_predict.majority_vote,
  grades.pat_id,
  grades.proc_ord_year
from
  lab.grader_table_with_metadata_project_independent grades
  join lab.maryam_nlp_combined nlp_predict on grades.proc_ord_id = nlp_predict.proc_ord_id
where
  grade = 0
  and nlp_predict.bert = nlp_predict.biobert
  and nlp_predict.bert = nlp_predict.clinbert
  and nlp_predict.bert = nlp_predict.radbert
  and nlp_predict.bert = 2
  and grades.proc_ord_year = 2024
'''

In [ ]:
df_conflicting_reports = client.query(q_conflicting_reports).to_dataframe()
l_conflicting_proc_ids = df_conflicting_reports['proc_ord_id'].values
l_header = ['proc_ord_id', 'orig_grader', 'reason_for_0', 'jenna_new_grade', 'correct_grader']
l_report_reasons = []

# Replace with the path to your .json file containing a dictionary of all the phrases you'd like to be highlighted
phrasesToHighlightFn = "../code/phrases_to_highlight.json"

# Load the dictionary of phrases to highlight in certain colors 
with open(phrasesToHighlightFn, 'r', encoding='utf-8') as f:
    to_highlight = json.load(f)

for conflicting_proc_id in l_conflicting_proc_ids:
    clear_output()
    orig_grader = print_report_experimental(conflicting_proc_id, client, to_highlight)
    reason_for_zero = str(input("Why does Jenna think was this report given a 0: "))
    new_grade_jenna = str(input("What grade would you give it now: "))
    ruling = str(input("Who was correct, models or human? "))
    l_report_reasons.append([conflicting_proc_id, orig_grader, reason_for_zero, new_grade_jenna, ruling])

In [ ]:
df_report_eval = pd.DataFrame(l_report_reasons, columns=l_header)
fn_out = '/home/youngjm/arcus/shared/nlp_models_vs_graders/2024_models2_graders0.csv'
df_report_eval.to_csv(fn_out, sep='\t')

In [ ]:
## Looking at the table
fn_out = '/home/youngjm/arcus/shared/nlp_models_vs_graders/2024_models2_graders0.csv'
df = pd.read_csv(fn_out, sep='\t')
df = df.drop(columns=["Unnamed: 0"])

In [ ]:
df